## Bayesian methods of hyperparameter optimization

In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

You will need to install the Bayesian optimization module. Running a cell with an exclamation point in the beginning of the command will run it as a shell command — please do this to install this module from our notebook in the cell below.

In [1]:
! pip install bayesian-optimization

In [2]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import lightgbm
from bayes_opt import BayesianOptimization


In [3]:
import os
os.listdir()

['.DS_Store',
 'flight_delays_test.csv',
 'Bayesian_optimization_case_study.ipynb',
 'flight_delays_train.csv',
 '.idea']

## How does Bayesian optimization work?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Let's look at a simple example

The first step is to create an optimizer. It uses two items:
* function to optimize
* bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.

Here we define our simple function we want to optimize.

In [4]:
def simple_func(a, b):
    return a + b

Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.

In [5]:
optimizer = BayesianOptimization(
    f=simple_func,
    pbounds={'a': (1, 3),'b': (4, 7)},
    allow_duplicate_points=True)

These are the main parameters of this function:

* **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.

In [6]:
optimizer.maximize(init_points=3, n_iter=2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
| 1         | 8.613     | 2.036     | 6.577     |
| 2         | 7.413     | 1.84      | 5.573     |
| 3         | 6.547     | 1.079     | 5.468     |
| 4         | 9.281     | 2.393     | 6.888     |
| 5         | 9.422     | 3.0       | 6.422     |


Great, now let's print the best parameters and the associated maximized target.

In [7]:
print(optimizer.max['params']);optimizer.max['target']

{'a': 3.0, 'b': 6.421704691399312}


9.421704691399313

## Test it on real data using the Light GBM

The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [8]:
train_df = pd.read_csv('flight_delays_train.csv')
test_df = pd.read_csv('flight_delays_test.csv')

**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**

In [9]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**

In [10]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


Notice, `DepTime` is the departure time in a numeric representation in 2400 hours.

In [11]:
DepTime = train_df['DepTime'].unique()
DepTime_min = [int(str(time)[-2:]) for time in DepTime]
sum(np.array(DepTime_min)>=60)

0

In [12]:
sum(np.array(DepTime)>2400)

16

In [13]:
train_df[train_df.DepTime > 2400].head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
8189,c-6,c-14,c-2,2435,EV,CVG,AVL,275,Y
20766,c-5,c-31,c-2,2534,EV,ATL,HSV,151,Y
27391,c-3,c-23,c-4,2505,EV,ATL,AGS,143,Y
44332,c-7,c-15,c-5,2440,EV,ATL,SHV,552,Y
45796,c-8,c-18,c-4,2447,EV,ATL,JAN,341,Y


In [14]:
train_df = train_df[train_df.DepTime <= 2400].copy()

In [15]:
train_df['dep_delayed_15min'] = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0})

 **<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**

In [16]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,0
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,0
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,0
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,0
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,1


## Feature Engineering
Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.

In [17]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')                               # use astype('uint8') change True/False to 1/0
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')  # use astype('uint8') change True/False to 1/0
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')                                # use astype('uint8') change True/False to 1/0
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')                    # use astype('int32') change True/False to 1/0
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')                     # use astype('int32') change True/False to 1/0
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')                     # use astype('int32') change True/False to 1/0
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')                   # use astype('int32') change True/False to 1/0
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int)                 # use astype(int) change True/False to 1/0
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)                  # use astype(int) change True/False to 1/0
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

Concatenate the training and testing dataframes.


In [18]:
full_df = pd.concat([train_df, test_df])
full_df = feature_eng(full_df)

In [19]:
full_df.isna().sum()

Month                            0
DayofMonth                       0
DayOfWeek                        0
UniqueCarrier                    0
Origin                           0
Dest                             0
Distance                         0
dep_delayed_15min           100000
flight                           0
begin_of_month                   0
midddle_of_month                 0
end_of_month                     0
hour                             0
morning                          0
day                              0
evening                          0
night                            0
winter                           0
spring                           0
summer                           0
autumn                           0
holiday                          0
weekday                          0
airport_dest_per_month           0
airport_origin_per_month         0
airport_dest_count               0
airport_origin_count             0
carrier_count                    0
carrier_count_per mo

Apply the earlier defined feature engineering functions to the full dataframe.

In [20]:
for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])

In [21]:
full_df.nunique()

Month                         12
DayofMonth                    31
DayOfWeek                      7
UniqueCarrier                 23
Origin                       307
Dest                         307
Distance                    1390
dep_delayed_15min              2
flight                      5048
begin_of_month                 2
midddle_of_month               2
end_of_month                   2
hour                          25
morning                        2
day                            2
evening                        2
night                          2
winter                         2
spring                         2
summer                         2
autumn                         2
holiday                        2
weekday                        2
airport_dest_per_month       396
airport_origin_per_month     388
airport_dest_count           228
airport_origin_count         221
carrier_count                 23
carrier_count_per month      248
deptime_cos                 1315
deptime_si


Split the new full dataframe into X_train and X_test.

In [22]:
train_df_new = full_df[:train_df.shape[0]]
test_df_new= full_df[train_df.shape[0]:]

In [23]:
train_df_new.head()

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,flight,begin_of_month,...,airport_origin_per_month,airport_dest_count,airport_origin_count,carrier_count,carrier_count_per month,deptime_cos,deptime_sin,flightUC,DestUC,OriginUC
0,8,21,7,1,19,82,732,0.0,171,0,...,1016,8290,11375,18024,1569,0.343660,-0.939094,265,494,67
1,4,20,3,19,226,180,834,0.0,3986,0,...,105,3523,1390,13069,1094,-0.612907,-0.790155,6907,1085,1441
2,9,2,5,21,239,62,416,0.0,4091,1,...,136,2246,1747,11737,977,-0.835807,-0.549023,7064,359,1518
3,11,25,6,16,81,184,872,0.0,1304,0,...,514,1785,6222,15343,1242,-0.884988,0.465615,2258,1122,484
4,10,7,6,20,182,210,423,1.0,2979,1,...,226,687,2571,30958,2674,0.073238,-0.997314,5144,1313,1103


In [24]:
test_df_new.head()

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,flight,begin_of_month,...,airport_origin_per_month,airport_dest_count,airport_origin_count,carrier_count,carrier_count_per month,deptime_cos,deptime_sin,flightUC,DestUC,OriginUC
0,7,25,3,22,198,222,598,NaN,3228,0,...,22,5571,221,6029,518,-0.039260,0.999229,5514,1432,1190
1,4,17,2,20,159,132,1235,NaN,2523,0,...,427,1513,4932,30958,2578,-0.355923,0.934515,4285,766,950
2,12,2,7,13,122,213,577,NaN,1955,1,...,27,9877,339,14424,1194,-0.133121,0.991100,3307,1340,714
3,3,25,7,20,49,188,377,NaN,694,0,...,253,571,3029,30958,2570,-0.467930,-0.883766,1183,1145,287
4,6,6,3,18,213,276,258,NaN,3697,1,...,849,1763,9823,13311,1125,-0.697790,-0.716302,6348,1829,1331


Create a list of the categorical features.

In [25]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

Let's build a light GBM model to test the bayesian optimizer.

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [26]:
from sklearn.model_selection import train_test_split

X = train_df_new.drop('dep_delayed_15min', axis=1)
y = train_df_new['dep_delayed_15min']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.30, random_state=42)

In [27]:
lgtrain = lightgbm.Dataset(X_train, y_train, categorical_feature=categorical_features, free_raw_data=False)

In [28]:
def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : "auc",
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "feature_pre_filter": False}

    cv_result = lightgbm.cv(params=params,
                            train_set=lgtrain,
                            num_boost_round=1000,
                            stratified=True,
                            nfold=3)
    return cv_result['valid auc-mean'][-1]

Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [29]:
lgbBO = BayesianOptimization(f=lgb_eval, pbounds={'num_leaves': (25, 4000),
                                                  'max_depth': (5, 63),
                                                  'lambda_l2': (0.0, 0.05),
                                                  'lambda_l1': (0.0, 0.05),
                                                  'min_child_samples': (50, 10000),
                                                  'min_data_in_leaf': (100, 2000)})

In [ ]:
lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
| 1         | 0.733     | 0.04892   | 0.02947   | 22.78     | 5.682e+03 | 1.104e+03 | 2.024e+03 |
| 2         | 0.7367    | 0.03834   | 0.03857   | 20.32     | 4.986e+03 | 1.897e+03 | 2.497e+03 |
| 3         | 0.7362    | 0.009514  | 0.03317   | 12.08     | 3.672e+03 | 1.249e+03 | 563.6     |
| 4         | 0.7328    | 0.0471    | 0.04525   | 33.99     | 4.898e+03 | 1.164e+03 | 2.608e+03 |
| 5         | 0.7365    | 0.03901   | 0.01233   | 47.95     | 7.984e+03 | 1.958e+03 | 629.4     |
| 6         | 0.7369    | 0.02775   | 0.03885   | 17.19     | 2.776e+03 | 1.832e+03 | 2.226e+03 |
| 7         | 0.7082    | 0.002291  | 0.04103   | 15.92     | 396.9     | 662.4     | 3.162e+03 |
| 8         | 0.736     | 0.006403  | 0.03412   | 42.17     | 9.939e+03 | 1.966e+03 | 3.742e+03 |
| 9         | 0.6957    | 0.01423   | 0.01937   | 8.699     | 9.923e+03 | 119.7     | 198.1     |
| 10        | 0.7365    | 0.04182   | 0.02257   | 22.99     | 7.499e+03 | 1.95e+03  | 3.755e+03 |
| 11        | 0.7366    | 0.04948   | 0.02525   | 48.22     | 5.867e+03 | 1.974e+03 | 181.4     |
| 12        | 0.7365    | 0.0277    | 0.03409   | 41.23     | 2.663e+03 | 1.861e+03 | 2.383e+03 |
=================================================================================================

In [ ]:
lgbBO.max


{'target': 0.7369473908252746,
 'params': {'lambda_l1': 0.02774976553924449,
  'lambda_l2': 0.03884635036275905,
  'max_depth': 17.191792483278753,
  'min_child_samples': 2775.6558710962963,
  'min_data_in_leaf': 1831.602834072356,
  'num_leaves': 2225.829374770909}}